In [ ]:
import copy
import json
import os
import urllib
from datetime import datetime
import boto3
import json

from sqlalchemy import create_engine, MetaData, Table, select, and_, func
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.automap import automap_base

import numpy as np
import pandas as pd


from utils import get_matching_s3_keys

In [ ]:
aws_credentials = json.load(open(os.environ["AWS_CREDENTIALS"]))
s3_client = boto3.client('s3', aws_access_key_id=aws_credentials["aws_access_key_id"],
                         aws_secret_access_key=aws_credentials["aws_secret_access_key"],
                         region_name="eu-west-1")

annotations_s3_bucket = 'aquabyte-annotations'

In [ ]:
sql_credentials = json.load(open(os.environ["SQL_CREDENTIALS"]))
sql_engine = create_engine("postgresql://{}:{}@{}:{}/{}".format(sql_credentials["user"], sql_credentials["password"],
                           sql_credentials["host"], sql_credentials["port"],
                           sql_credentials["database"]))

Session = sessionmaker(bind=sql_engine)
session = Session()



In [ ]:
Base = automap_base()
Base.prepare(sql_engine, reflect=True)
Enclosure = Base.classes.enclosures
Calibration = Base.classes.calibrations
GtsfDataCollection = Base.classes.gtsf_data_collections
StereoFramePair = Base.classes.stereo_frame_pairs


In [ ]:
def convert_to_world_point(x, y, d, pixel_count_width, 
                           pixel_count_height, image_sensor_width, 
                           image_sensor_height, focal_length):
    """ from pixel coordinates to world coordinates """
    
    image_center_x = pixel_count_width / 2.0  
    image_center_y = pixel_count_height / 2.0
    px_x = x - image_center_x
    px_z = image_center_y - y

    sensor_x = px_x * (image_sensor_width / 4096)
    sensor_z = px_z * (image_sensor_height / 3000)

    world_y = d
    world_x = (world_y * sensor_x) / focal_length
    world_z = (world_y * sensor_z) / focal_length
    return [world_x, world_y, world_z]



def depth_from_disp(disp, focal_length_pixel, baseline):
    """ calculate the depth of the point based on the disparity value """
    depth = focal_length_pixel*baseline / np.array(disp)
    return depth


def euclidean_distance(p1, p2):
    return ((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2 + (p1[2] - p2[2])**2)**0.5

In [ ]:
annotations_f = '/app/data/cloud_data_service/annotations.json'

def cogito_main(s3_client, annotations_s3_bucket, image_s3_bucket):
    """ every hour check s3 folder for new files"""
    generator = get_matching_s3_keys(s3_client,
                                     annotations_s3_bucket,
                                     prefix='cogito/gtsf_keypoint_annotations',
                                     suffix='.json')

    for key in generator:
        print(key)
        date = str(key.replace('.json', '').split('/')[-1])
        # check if it's already been processed -- if it has, continue
        # logic to be added
        
        s3_client.download_file(annotations_s3_bucket, key, annotations_f)
        annotation_objects = json.load(open(annotations_f))
        
        # remove duplicates
        
        annotations_by_datarow_id = {}
        for obj in annotation_objects:
            datarow_id = obj['DataRow ID']
            if datarow_id not in list(annotations_by_datarow_id.keys()):
                annotations_by_datarow_id[datarow_id] = obj
            else:
                if obj['Created At'] > annotations_by_datarow_id[datarow_id]:
                    annotations_by_datarow_id[datarow_id] = obj
                    
        anns = annotations_by_datarow_id.values()
        
        # get body parts list
        body_parts = []
        for k, v in anns[0]['Label'].items():
            body_parts.append(k)
            
        # create keypoints dataframe
        keypoints_df = pd.DataFrame()

        gtsf_data_collections = session.query(GtsfDataCollection).filter(GtsfDataCollection.date == date).all()
        
        fish_ids = []
        for idx, obj in enumerate(anns):
            if obj['Label'] == 'Skip':
                continue

            # get fish_id
            image_url = str(obj['Labeled Data'])
            s3_bucket_key = image_url[image_url.index(image_s3_bucket):]
            s3_key = os.path.join(*s3_bucket_key.split('/')[1:])
            fish_id = image_url.split('/')[-3]
            if not fish_id in fish_ids:
                fish_ids.append(fish_id)

            # get image file name and epoch
            image_f_name = image_url.split('/')[-1]
            epoch = int(image_f_name.replace('.jpg', '').split('_')[-1])

            camera = str(image_f_name.split('_')[0])

            for body_part in body_parts:
                kp_dict = obj['Label'][body_part][0]['geometry']
                kp = (kp_dict['x'], kp_dict['y'])

                row = {
                    'fish_id': fish_id,
                    'epoch': epoch,
                    'image_s3_key': s3_key,
                    'body_part': body_part,
                    'camera': camera,
                    'keypoint': kp

                }

                keypoints_df = keypoints_df.append(row, ignore_index=True)

        
        
        # get camera details
        gtsf_data_collection = session.query(GtsfDataCollection) \
                                      .filter(GtsfDataCollection.gtsf_fish_identifier==fish_ids[0]) \
                                      .one()

        calibration = session.query(Calibration) \
                     .filter(Calibration.enclosure_id == gtsf_data_collection.enclosure_id) \
                     .order_by(Calibration.utc_timestamp.desc()) \
                     .first()
                
        enclosure = session.query(Enclosure).get(calibration.enclosure_id)
            
        
        focal_length = float(calibration.predicted_focal_length_mm) / (1e3)
        baseline = float(calibration.predicted_baseline_mm) / (1e3)
        pixel_size_m = float(enclosure.pixel_width_um) / (1e6)
        focal_length_pixel = focal_length / pixel_size_m
        image_sensor_width = float(enclosure.sensor_width_mm) / (1e3)
        image_sensor_height = float(enclosure.sensor_height_mm) / (1e3)
        pixel_count_width = enclosure.image_num_pixels_width
        pixel_count_height = enclosure.image_num_pixels_height
        
        # create stereo frame pairs df
        
        stereo_frame_pairs_df = pd.DataFrame()
        for fish_id in fish_ids:
            gtsf_data_collection = session.query(GtsfDataCollection) \
                                          .filter(GtsfDataCollection.gtsf_fish_identifier==fish_id) \
                                          .one()

            calibration = session.query(Calibration) \
                                 .filter(Calibration.enclosure_id==gtsf_data_collection.enclosure_id) \
                                 .one()

            fish_id_mask = keypoints_df.fish_id == fish_id
            epochs = keypoints_df[fish_id_mask].epoch.unique()
            for epoch in epochs:
                epoch_mask = keypoints_df.epoch == epoch
                row = {}
                row['gtsf_data_collection_id'] = gtsf_data_collection.id
                row['gtsf_fish_identifier'] = fish_id
                row['epoch'] = epoch

                row['left_image_s3_key'] = keypoints_df[fish_id_mask \
                                                        & epoch_mask \
                                                        & (keypoints_df.camera == 'left')
                                                       ].image_s3_key.iloc[0]

                row['right_image_s3_key'] = keypoints_df[fish_id_mask \
                                                        & epoch_mask \
                                                        & (keypoints_df.camera == 'right')
                                                       ].image_s3_key.iloc[0]

                left_keypoints, right_keypoints, world_keypoints = {}, {}, {}

                for body_part in body_parts:
                    left_row = keypoints_df[
                        (keypoints_df.epoch == epoch) & (keypoints_df.camera == 'left') & (keypoints_df.body_part == body_part)
                    ].iloc[0]

                    lkp = left_row['keypoint']
                    left_keypoints[body_part] = lkp

                    right_row = keypoints_df[
                        (keypoints_df.epoch == epoch) & (keypoints_df.camera == 'right') & (keypoints_df.body_part == body_part)
                    ].iloc[0]

                    rkp = right_row['keypoint']
                    right_keypoints[body_part] = rkp

                    d = abs(lkp[0] - rkp[0])

                    # compute world key point
                    depth = depth_from_disp(d, focal_length_pixel, baseline)
                    wkp = convert_to_world_point(lkp[0], lkp[1], depth, pixel_count_width, 
                                                 pixel_count_height, image_sensor_width, 
                                                 image_sensor_height, focal_length)

                    world_keypoints[body_part] = wkp

                row['left_keypoints'] = left_keypoints
                row['right_keypoints'] = right_keypoints
                row['world_keypoints'] = world_keypoints

                stereo_frame_pairs_df = stereo_frame_pairs_df.append(row, ignore_index=True)
                
        # add stereo frame pairs to database
        
        for idx, row in stereo_frame_pairs_df.iterrows():
            stereo_frame_pair = StereoFramePair(
                gtsf_data_collection_id=row['gtsf_data_collection_id'],
                gtsf_fish_identifier=row['gtsf_fish_identifier'],
                date=date,
                epoch=int(row['epoch']),
                left_image_s3_key=row['left_image_s3_key'],
                right_image_s3_key=row['right_image_s3_key'],
                image_s3_bucket=image_s3_bucket,
                left_image_keypoint_coordinates=json.dumps(row['left_keypoints']),
                right_image_keypoint_coordinates=json.dumps(row['right_keypoints']),
                world_keypoint_coordinates=json.dumps(row['world_keypoints']),
                annotations_project_name=annotation_objects[0]['Project Name'],
                annotations_file_s3_key=key,
                annotations_s3_bucket=annotations_s3_bucket
            )
            
            session.add(stereo_frame_pair)
            session.commit()




In [ ]:
cogito_main(s3_client, 'aquabyte-annotations', 'aquabyte-groundtruths')

In [ ]:
wkp = df[df.gtsf_fish_identifier == '190226010001'].world_keypoints.iloc[0]

In [ ]:
session.rollback()